# SW Experiments

### Experimental variable is da_ds, varying in steps by 1% off each side from a steady state

Need:
- larger domain/mesh loaded in
- smb set up
  

## Questions
- How long does each run need to be?
- Does the domain need to run all the way back to the divide?
- When do we do the full region?
- Should smb be in each timestep or evolve from the initialization?

In [ ]:
import argparse
import subprocess
import tqdm
import numpy as np
from numpy import pi as π
import xarray
import firedrake
from firedrake import assemble, Constant, exp, max_value, inner, grad, dx, ds, dS
import icepack
from icepack2.constants import (
    glen_flow_law as n,
    weertman_sliding_law as m,
    ice_density as ρ_I,
    water_density as ρ_W,
)
from icepack2 import model

### Controls for SMB and time stepping

In [ ]:
timesteps_per_year = 1
final_time = 4000
calving = False # When running to sealevel
melt_rate = 3e3
mask_smoothing_length = 5e3
snes_max_it = 200
snes_rtol = 1e-5

### Read in the starting data

In [ ]:
with firedrake.CheckpointFile('mystery-glacier-simulation.h5', "r") as chk:
    mesh = chk.load_mesh()
    q = chk.load_function(mesh, name="log_friction")
    τ_c = chk.h5pyfile.attrs["mean_stress"]
    u_c = chk.h5pyfile.attrs["mean_speed"]

    timesteps = np.array(chk.h5pyfile["timesteps"])
    u = chk.load_function(mesh, name="velocity", idx=len(timesteps) - 1)
    h = chk.load_function(mesh, name="thickness", idx=len(timesteps) - 1)

### Setting up the space

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2) # Changed from 1
Δ = firedrake.FunctionSpace(mesh, "DG", 1)
V = firedrake.VectorFunctionSpace(mesh, "CG", 1) # Changed from 1
Σ = firedrake.TensorFunctionSpace(mesh, "DG", 0, symmetry=True)
T = firedrake.VectorFunctionSpace(mesh, "DG", 0)
Z = V * Σ * T

In [ ]:
u_in = firedrake.project(u, V)
q = firedrake.project(q, Q)

z = firedrake.Function(Z)
z.sub(0).assign(u_in)

In [ ]:
# Read the thickness and bed data
bedmachine = xarray.open_dataset(icepack.datasets.fetch_bedmachine_greenland())
b = icepack.interpolate(bedmachine["bed"], Q)
h = firedrake.project(h, Δ)
s = firedrake.project(max_value(b + h, (1 - ρ_I / ρ_W) * h), Δ)

In [ ]:
# Set up the momentum balance equation and solve
A = icepack.rate_factor(Constant(260))
ε_c = Constant(A * τ_c ** n)
print(f"τ_c: {1000 * float(τ_c):.1f} kPa")
print(f"ε_c: {1000 * float(ε_c):.1f} (m / yr) / km")
print(f"u_c: {float(u_c):.1f} m / yr")

In [ ]:
fns = [
    model.viscous_power,
    model.friction_power,
    #model.calving_terminus,
    model.momentum_balance,
]

In [ ]:
u, M, τ = firedrake.split(z)
fields = {
    "velocity": u,
    "membrane_stress": M,
    "basal_stress": τ,
    "thickness": h,
    "surface": s,
}

h_min = Constant(10.0)
rfields = {
    "velocity": u,
    "membrane_stress": M,
    "basal_stress": τ,
    "thickness": max_value(h_min, h),
    "surface": s,
}

rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c**n,
    "sliding_exponent": m,
    "sliding_coefficient": u_c / τ_c**m * exp(m * q),
}

linear_rheology = {
    "flow_law_exponent": 1,
    "flow_law_coefficient": ε_c / τ_c,
    "sliding_exponent": 1,
    "sliding_coefficient": u_c / τ_c * exp(q),
}

In [ ]:
L_1 = sum(fn(**rfields, **linear_rheology) for fn in fns)
F_1 = firedrake.derivative(L_1, z)
J_1 = firedrake.derivative(F_1, z)

In [ ]:
L = sum(fn(**fields, **rheology) for fn in fns)
F = firedrake.derivative(L, z)

In [ ]:
L_r = sum(fn(**rfields, **rheology) for fn in fns)
F_r = firedrake.derivative(L_r, z)
J_r = firedrake.derivative(F_r, z)
α = firedrake.Constant(0.0)
J = J_r + α * J_1

In [ ]:
inflow_ids = [1]
bc_in = firedrake.DirichletBC(Z.sub(0), u_in, inflow_ids)
outflow_ids = [2]
bc_out = firedrake.DirichletBC(Z.sub(0), Constant((0.0, 0.0)), outflow_ids)
bcs = [bc_in, bc_out]

In [ ]:
qdegree = int(max(m, n)) + 2
problem_params = {
    "form_compiler_parameters": {"quadrature_degree": qdegree},
    "bcs": bcs,
}
solver_params = {
    "solver_parameters": {
        "snes_monitor": None,
        #"snes_converged_reason": None,
        "snes_stol": 0.0,
        "snes_rtol": 1e-6,
        "snes_linesearch_type": "nleqerr",
        "snes_max_it": 200,
        "snes_divergence_tolerance": -1,
        "snes_type": "newtonls",
        "ksp_type": "gmres",
        "pc_type": "lu",
        # If the linear solver crashes, try "umfpack" instead of "mumps"
        "pc_factor_mat_solver_type": "mumps",
    },
}
firedrake.solve(F_1 == 0, z, **problem_params, **solver_params)

In [ ]:
u_problem = firedrake.NonlinearVariationalProblem(F, z, J=J, **problem_params)
u_solver = firedrake.NonlinearVariationalSolver(u_problem, **solver_params)
u_solver.solve()

### Setting up SMB Initalization
- Is any of this right
- Vary ds_ds
- batching runs?

In [ ]:
da_ds = Constant(2.32 * 1e-3) #2.25

a_val = -1.8
a_0 = Constant(a_val) #-2.3

ela = - a_val / da_ds
print(ela)

smb = 0.917 * (a_0 + da_ds * s)

In [ ]:
print(s.dat.data_ro.max())
print(s.dat.data_ro.mean())

response_time = h.dat.data_ro.mean() / -a_val
print(f'Response time is ~{response_time} years')

In [ ]:
# Create a solver for a smooth ice mask field
if calving == True:
    α = Constant(args.mask_smoothing_length)
    μ = firedrake.Function(Q)
    χ = firedrake.conditional(h > 0, 1, 0)
    J = 0.5 * ((μ - χ)**2 + α**2 * inner(grad(μ), grad(μ))) * dx
    bcs = [
        firedrake.DirichletBC(Q, Constant(1.0), (1,)),
        firedrake.DirichletBC(Q, Constant(0.0), (3,)),
    ]
    F = firedrake.derivative(J, μ)
    μ_problem = firedrake.NonlinearVariationalProblem(F, μ, bcs)
    μ_solver = firedrake.NonlinearVariationalSolver(μ_problem)
    μ_solver.solve()

In [ ]:
# Create the accumulation/ablation function, which is a sum of SMB and calving
# losses (if any)
t = Constant(0.0)
if calving == True:
    m = Constant(args.melt_rate)  # melt rate in m/yr
    φ = firedrake.min_value(0, firedrake.cos(2 * π * t))
    calving = m * (1 - μ) * φ
    a = smb + calving
else:
    a = smb

In [ ]:
# Set up the mass balance equation
h_n = h.copy(deepcopy=True)
h0 = h.copy(deepcopy=True)
φ = firedrake.TestFunction(h.function_space())
dt = Constant(1.0 / 96)
flux_cells = ((h - h_n) / dt * φ - inner(h * u, grad(φ)) - a * φ) * dx
ν = firedrake.FacetNormal(mesh)
f = h * max_value(0, inner(u, ν))
flux_facets = (f("+") - f("-")) * (φ("+") - φ("-")) * dS
flux_in = h0 * firedrake.min_value(0, inner(u, ν)) * φ * ds
flux_out = h * max_value(0, inner(u, ν)) * φ * ds
G = flux_cells + flux_facets + flux_in + flux_out
h_problem = firedrake.NonlinearVariationalProblem(G, h)
h_solver = firedrake.NonlinearVariationalSolver(h_problem)

### Run the simulation

In [ ]:
num_steps = int(final_time * timesteps_per_year) + 1

dh_max = np.zeros(num_steps) * np.nan
hs = [h.copy(deepcopy=True)]
h_c = Constant(5.0)

with firedrake.CheckpointFile('mystery-glacier-simulation.h5', "w") as chk:
    u, M, τ = z.subfunctions
    chk.save_function(h, name="thickness", idx=0)
    chk.save_function(u, name="velocity", idx=0)
    chk.save_function(M, name="membrane_stress", idx=0)
    chk.save_function(τ, name="basal_stress", idx=0)
    if calving == True:
        chk.save_function(μ, name="ice_mask", idx=0)

    timesteps = np.linspace(0.0, final_time, num_steps)
    for step in tqdm.trange(num_steps):
        t.assign(t + dt)
        if calving == True:
            μ_solver.solve()
 
        h_solver.solve()
        h.interpolate(firedrake.conditional(h < h_c, 0, h))
        h_n.assign(h)
        hs.append(h.copy(deepcopy=True))
        dh_max[step] = h.dat.data_ro.max()
        
        s.interpolate(max_value(b + h, (1 - ρ_I / ρ_W) * h))
        u_solver.solve()

        # Save the results to disk
        u, M, τ = z.subfunctions
        chk.save_function(h, name="thickness", idx=step + 1)
        chk.save_function(u, name="velocity", idx=step + 1)
        chk.save_function(M, name="membrane_stress", idx=step + 1)
        chk.save_function(τ, name="basal_stress", idx=step + 1)
        if calving == True:
            chk.save_function(μ, name="ice_mask", idx=step + 1)

    chk.save_function(q, name="log_friction")
    chk.h5pyfile.attrs["mean_stress"] = τ_c
    chk.h5pyfile.attrs["mean_speed"] = u_c
    chk.h5pyfile.create_dataset("timesteps", data=timesteps)

### Pictures because I can't read the raw outputs

In [ ]:
# Difference plot 
δh = firedrake.project(hs[-1] - hs[0], hs[0].function_space())
change_max = δh.dat.data_ro.max()
change_min = δh.dat.data_ro.min()

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(δh, 
                             num_sample_points=1, 
                             shading="flat", 
                             cmap="RdBu",
                             vmin = -300, vmax = 300, 
                             axes=axes)

fig.colorbar(colors);